In [1]:
cd ..

/home/noahb/fv3net


In [2]:
import xarray as xr
from src.data.advect import *

In [3]:
ds = xr.open_zarr("/home/noahb/data/2019-08-07_from-2019-07-17-GFDL_FV3_DYAMOND_0.25deg_15minute.zarr")
data_3d = xr.open_zarr("/home/noahb/data/2019-07-17-GFDL_FV3_DYAMOND_0.25deg_15minute/3d.zarr/").merge(ds)
data_2d = xr.open_zarr("/home/noahb/data/2019-07-17-GFDL_FV3_DYAMOND_0.25deg_15minute/2d.zarr/")

h = height_on_model_levels(data_3d)
data_3d['z'] = h

In [4]:
mkdir -p data/interim

In [20]:
variables = [key for key in data_3d if not key.endswith('plev')]

chunked_data = data_3d[variables].chunk({'grid_xt': 256, 'grid_yt': 256, 'time': 1})

# delete chunks encoding following https://github.com/pydata/xarray/issues/2300
for key in chunked_data:
    try:
        del chunked_data[key].encoding['chunks']
    except:
        pass
    
chunked_data.to_zarr("data/interim/2019-07-17-FV3_DYAMOND_0.25deg_15minute_256x256_blocks.zarr", mode="w")